In [2]:
import requests
import json
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from datetime import datetime
from nba_api.stats.static import teams 
from functools import reduce
import mysql.connector
from config import conn_host, conn_database, conn_user, conn_password
from nba_api.stats.endpoints import leaguegamelog

In [10]:
first_season = 2008
last_season = 2022

now = time.time()

In [4]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
        db.close()
    except Exception as e:
        print(e)
    return resp

def execute_multiple_queries(queries):
    try:
        db = connect_to_db()
        mycursor = db.cursor()
        for query in queries:
            mycursor.execute(query)

        db.commit()
        db.close()
    except Exception as e:
        print(e)

In [5]:
def add_match_info_to_db(home_game, away_game, winner):
    query = (f"INSERT IGNORE INTO games (id, date, season, is_playoff, winner, " + 
                  "home_id, home_pts, home_fgm, home_fga, home_fg_pct, home_fg3m, home_fg3a, home_fg3_pct, home_ftm, home_fta, home_ft_pct, home_oreb, home_dreb, home_reb, home_ast, home_stl, home_blk, home_tov, home_pf," +
                  "away_id, away_pts, away_fgm, away_fga, away_fg_pct, away_fg3m, away_fg3a, away_fg3_pct, away_ftm, away_fta, away_ft_pct, away_oreb, away_dreb, away_reb, away_ast, away_stl, away_blk, away_tov, away_pf" +
                  f") VALUES ({home_game['GAME_ID']}, '{home_game['GAME_DATE']}', {get_season_year(home_game['SEASON_ID'])}, {home_game['IS_PLAYOFFS']}, '{winner}', " + 
                  f"{home_game['TEAM_ID']}, {home_game['PTS']}, {home_game['FGM']}, {home_game['FGA']}, {home_game['FG_PCT']}, {home_game['FG3M']}, {home_game['FG3A']}, {home_game['FG3_PCT']}, {home_game['FTM']}, {home_game['FTA']}, {home_game['FT_PCT']}, {home_game['OREB']}, {home_game['DREB']}, {home_game['REB']}, {home_game['AST']}, {home_game['STL']}, {home_game['BLK']}, {home_game['TOV']}, {home_game['PF']}, " +
                  f"{away_game['TEAM_ID']}, {away_game['PTS']}, {away_game['FGM']}, {away_game['FGA']}, {away_game['FG_PCT']}, {away_game['FG3M']}, {away_game['FG3A']}, {away_game['FG3_PCT']}, {away_game['FTM']}, {away_game['FTA']}, {away_game['FT_PCT']}, {away_game['OREB']}, {away_game['DREB']}, {away_game['REB']}, {away_game['AST']}, {away_game['STL']}, {away_game['BLK']}, {away_game['TOV']}, {away_game['PF']}" +
                  ")")
    return query
    
def add_team_to_db(team):
    query = (f"INSERT IGNORE INTO teams (id, name, abbreviation) VALUES ({team['id']}, '{team['full_name']}', '{team['abbreviation']}')")
    return query

def add_player_to_db(player_id, name):
    query = (f"INSERT IGNORE INTO players (id, name) VALUES ({player_id}, '{name}')")
    return query
    
def add_player_game_to_db(game):
    query = (f"INSERT IGNORE INTO playergames (team_id, player_id, game_id, minutes, pts, fgm, fga, fg_pct, fg3m, fg3a, fg3_pct, ftm, fta, ft_pct, oreb, dreb, reb, ast, stl, blk, tov, pf, plus_minus) " + 
                  f"VALUES ({game['TEAM_ID']}, {game['PLAYER_ID']}, {game['GAME_ID']}, {game['MIN']}, {game['PTS']}, {game['FGM']}, {game['FGA']}, {game['FG_PCT']}, {game['FG3M']}, {game['FG3A']}, {game['FG3_PCT']}, {game['FTM']}, {game['FTA']}, {game['FT_PCT']}, {game['OREB']}, {game['DREB']}, {game['REB']}, {game['AST']}, {game['STL']}, {game['BLK']}, {game['TOV']}, {game['PF']}, {game['PLUS_MINUS']})")
    return query

In [6]:
def get_season(season, season_type):
    season_i_teams = leaguegamelog.LeagueGameLog(season = str(season), season_type_all_star=season_type).get_data_frames()[0]
    season_i_players = leaguegamelog.LeagueGameLog(season = str(season), player_or_team_abbreviation = 'P', season_type_all_star=season_type).get_data_frames()[0]
    season_i_teams['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    season_i_players['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    return season_i_teams, season_i_players

def get_season_year(season_id):
    return int(str(season_id)[1:])

In [13]:
# Table creation queries
execute_query("CREATE TABLE IF NOT EXISTS teams (id INT NOT NULL, name VARCHAR(50) NOT NULL, abbreviation VARCHAR(4) NOT NULL, PRIMARY KEY (id))", False)
execute_query("CREATE TABLE IF NOT EXISTS games (id INT NOT NULL, date DATETIME NOT NULL, season INT NOT NULL, is_playoff BOOL NOT NULL, winner ENUM('H', 'A') NULL, " +
              "home_id INT NOT NULL, home_pts INT NULL, home_fgm INT NULL, home_fga INT NULL, home_fg_pct FLOAT NULL, home_fg3m INT NULL, home_fg3a INT NULL, home_fg3_pct FLOAT NULL, home_ftm INT NULL, home_fta INT NULL, home_ft_pct FLOAT NULL, home_oreb INT NULL, home_dreb INT NULL, home_reb INT NULL, home_ast INT NULL, home_stl INT NULL, home_blk INT NULL, home_tov INT NULL, home_pf INT NULL," +
              "away_id INT NOT NULL, away_pts INT NULL, away_fgm INT NULL, away_fga INT NULL, away_fg_pct FLOAT NULL, away_fg3m INT NULL, away_fg3a INT NULL, away_fg3_pct FLOAT NULL, away_ftm INT NULL, away_fta INT NULL, away_ft_pct FLOAT NULL, away_oreb INT NULL, away_dreb INT NULL, away_reb INT NULL, away_ast INT NULL, away_stl INT NULL, away_blk INT NULL, away_tov INT NULL, away_pf INT NULL," +
              "PRIMARY KEY (id), FOREIGN KEY(home_id) REFERENCES teams (id), FOREIGN KEY(away_id) REFERENCES teams (id))", False)
execute_query("CREATE TABLE IF NOT EXISTS players (id INT NOT NULL, name VARCHAR(100) NOT NULL, PRIMARY KEY (id))", False)
execute_query("CREATE TABLE IF NOT EXISTS playergames (team_id INT NOT NULL, player_id INT NOT NULL, game_id INT NOT NULL, minutes INT NULL, pts INT NULL, fgm INT NULL, fga INT NULL, fg_pct FLOAT NULL, fg3m INT NULL, fg3a INT NULL, fg3_pct FLOAT NULL, ftm INT NULL, fta INT NULL, ft_pct FLOAT NULL, oreb INT NULL, dreb INT NULL, reb INT NULL, ast INT NULL, stl INT NULL, blk INT NULL, tov INT NULL, pf INT NULL, plus_minus INT NULL, " + 
              "PRIMARY KEY (player_id, game_id), FOREIGN KEY(team_id) REFERENCES teams (id), FOREIGN KEY(player_id) REFERENCES players (id), FOREIGN KEY(game_id) REFERENCES games (id))", False)

In [11]:
pd.options.mode.chained_assignment = None  # default='warn'

seasons_teams = []
seasons_players = []
first_season_id = 20000 + first_season

print("Getting NBA Seasons Information...")
for i in range(first_season,last_season):
    season_i_teams, season_i_players = get_season(str(i), 'Regular Season')
    season_i_teams_playoffs, season_i_players_playoffs = get_season(str(i), 'Playoffs')
    
    season_i_teams = reduce(lambda left,right: pd.merge(left,right, how='outer'), [season_i_teams, season_i_teams_playoffs])
    season_i_players = reduce(lambda left,right: pd.merge(left,right, how='outer'), [season_i_players, season_i_players_playoffs])
    
    seasons_teams.append(season_i_teams)
    seasons_players.append(season_i_players)
    print("{}/{}".format(i,last_season))
    clear_output(wait=True)


dfs = []

print("Cleaning the data...")

season_games = reduce(lambda  left,right: pd.merge(left,right, how='outer'), seasons_teams)
season_games_plyrs = reduce(lambda  left,right: pd.merge(left,right, how='outer'), seasons_players)
season_games.dropna(subset=['FG_PCT','FT_PCT','FG3_PCT'], inplace=True)

season_games['GAME_ID'] = pd.to_numeric(season_games['GAME_ID'])
season_games_plyrs['GAME_ID'] = pd.to_numeric(season_games_plyrs['GAME_ID'])

season_games['GAME_DATE'] = pd.to_datetime(season_games['GAME_DATE'])
season_games_plyrs['GAME_DATE'] = pd.to_datetime(season_games_plyrs['GAME_DATE'])

season_games = season_games.sort_values(['GAME_DATE', 'GAME_ID'], ascending=[True, True]).reset_index(drop=True)

print('size', len(season_games.index))

Cleaning the data...
size 35658


In [12]:
season_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,IS_PLAYOFFS
0,22008,1610612739,CLE,Cleveland Cavaliers,20800001,2008-10-28,CLE @ BOS,L,240,29,...,41,16,8,2,22,26,85,-5,0,False
1,22008,1610612738,BOS,Boston Celtics,20800001,2008-10-28,BOS vs. CLE,W,240,33,...,36,16,10,0,17,28,90,5,0,False
2,22008,1610612741,CHI,Chicago Bulls,20800002,2008-10-28,CHI vs. MIL,W,240,35,...,40,27,9,6,19,18,108,13,0,False
3,22008,1610612749,MIL,Milwaukee Bucks,20800002,2008-10-28,MIL @ CHI,L,240,37,...,32,21,5,3,19,30,95,-13,0,False
4,22008,1610612757,POR,Portland Trail Blazers,20800003,2008-10-28,POR @ LAL,L,240,29,...,44,19,8,3,14,17,76,-20,0,False


In [84]:
teams_list = teams.get_teams()

teams_to_insert_queries = []

[teams_to_insert_queries.append(add_team_to_db(team)) for team in teams_list]

execute_multiple_queries(teams_to_insert_queries)

In [14]:
games_to_insert_queries = []
players_to_insert_queries = []
player_games_to_insert_queries = []

season_id = ''
for i, g in season_games.groupby(season_games.index // 2):
    clear_output(wait=True)
    print("{}/{}".format(i, len(season_games.index) // 2))
    if g.iloc[[0],:].iloc[0]['WL'] == None:
        break
        
    if '@' in g.iloc[[0],:].iloc[0]['MATCHUP']:
        away_game = g.iloc[0,:]
        home_game = g.iloc[1,:]
        winner = 'H' if g.iloc[1,:]['WL'] == 'W' else 'A'
    else:
        home_game = g.iloc[0,:]
        away_game = g.iloc[1,:]
        winner = 'H' if g.iloc[0,:]['WL'] == 'W' else 'A'
    
    game_players = season_games_plyrs.loc[season_games_plyrs['GAME_ID'] == home_game['GAME_ID']]
    game_players = game_players.replace({np.nan: 0})
    
    games_to_insert_queries.append(add_match_info_to_db(home_game, away_game, winner))
    
    for index, player in game_players.iterrows():
        players_to_insert_queries.append(add_player_to_db(player['PLAYER_ID'], player['PLAYER_NAME'].replace("'", "")))
        player_games_to_insert_queries.append(add_player_game_to_db(player))

17828/17829


In [15]:
execute_multiple_queries(games_to_insert_queries)
execute_multiple_queries(players_to_insert_queries)
execute_multiple_queries(player_games_to_insert_queries)

In [16]:
games_df = execute_query(f"SELECT m.id, m.date, m.season, ht.name as home_team, at.name as away_team, m.home_pts, m.away_pts FROM games AS m INNER JOIN teams AS ht ON (m.home_id = ht.id) INNER JOIN teams AS at ON (m.away_id = at.id) ORDER BY m.date DESC")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
games_df.head()

,id,date,season,home_team,away_team,home_pts,away_pts
0,42100406,2022-06-16,2021,Boston Celtics,Golden State Warriors,90,103
1,42100405,2022-06-13,2021,Golden State Warriors,Boston Celtics,104,94
2,42100404,2022-06-10,2021,Boston Celtics,Golden State Warriors,97,107
3,42100403,2022-06-08,2021,Boston Celtics,Golden State Warriors,116,100
4,42100402,2022-06-05,2021,Golden State Warriors,Boston Celtics,107,88
